In [3]:
from pwn import *
import re

```python3
#!/usr/bin/env pypy3

import os
from pydoc import plain
from sys import byteorder
from Crypto.Cipher import AES
from Crypto.Util import Counter
import hashlib

# Create a secret.py file with a variable `FLAG` for local testing :)
from secret import FLAG

secret_key = os.urandom(16)

def encrypt(plaintext, counter):
    m = hashlib.sha256()
    m.update(counter.to_bytes(8, byteorder="big"))

    alg = AES.new(secret_key, AES.MODE_CTR, nonce=m.digest()[0:8])
    ciphertext = alg.encrypt(plaintext)

    return ciphertext.hex()


def main():
    print("DES is broken, long live the secure AES encryption!")
    print("Give me a plaintext and I'll encrypt it a few times for you. For more security of course!")

    try:
        plaintext = bytes.fromhex(input("Enter some plaintext (hex): "))
    except ValueError:
        print("Please enter a hex string next time.")
        exit(0)
    
    for i in range(0, 255):
        print(f"Ciphertext {i:03d}: {encrypt(plaintext, i)}")
    
    print("Flag:", encrypt(FLAG.encode("ascii"), int.from_bytes(os.urandom(1), byteorder="big")))

if __name__ == "__main__":
    main()
```

We know AES-CTR encrypted like a stream cipher (no padding) by:

$c_i = Enc(k,nonce||ctr) \oplus m_i$

The flag and the choosen plaintext are encrypted with the **same key** and have counters **starting at 0**. However, the nonce is unkown for the encrpyted flag, but one of the send ciphertexts $c_i$ will use that nonce to encrypt our plaintext. Additionally, we can find $k_i = Enc(k,nonce||ctr)$ by sending $0^{\text{len(flag)}}$ to the oracle. Because we know that the flag starts with $CSCG$, we can xor $CSCG$ to the flag to get $k_{flag}$ for the first 4 bytes of the flag. We then search through all ciphertexts/keystreams ($c_i==k_i$) for all $i \in \{0,255\}$ until we find one where the first 4 bytes match to $k_{flag}$.

In [2]:
flag = "3a851b8e1c082cb4d65cd6ac5f4b94811c27658394d7d6f7ab3649d087a790126b447c2faadd47ee4f5fdb3393ed395ce063a1b84dbe962a0705687cd849026fa4ddcab7d7"
payload = bytes([0] * len(bytes.fromhex(flag))) 
payload.hex() 

'000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'

In [46]:
data = """Ciphertext 000: 0cbb38da49c0be4357117f7dc86d5284210c85eb7b1b20fbbb15f8298ea7e3b362a3a8266aa50522b16ef12d411497161017515e5fd4111f0d016412c1548b8bda9fc7bb63
Ciphertext 001: 4083e0598bff3f619b83190fc17dd2a078274a08ef66473b70374f898acadbe2ffdd1afa11b98064e5f48da3cc8897203d4fcae50b7ce16a2eda0b7158400121f3c8854e41
Ciphertext 002: c761b32128820ec0c5af148178edf65b3af221643145d35c321887e4b3914a53e67e03947738848b6573eb2370103add101e5c0c0007a147b547bb2739f73982b87393df65
Ciphertext 003: a883b5dac32ad6cd7dc3d9d0a9271d2d8e13314238c13e5967a88ed48b0c16540d6c6f5568e11fd20b8de51f82e9c6ac04a8aaa4e415f090699a17d8ef36b27f4222cd5bde
Ciphertext 004: 5797dbdade953fef25156d7038b3013797dd25e9a9c0d93ff6b2e680a0da94b3e618949efcd427ef098d7c1279bc9e87f96f14f30158e9b22399fd859e7957ba56735a904b
Ciphertext 005: f5d8394ce176a095194c2ac3ab797b49448393cdcdccec1a4afa55d00a8ae46ce4733eb12917a337b152a4e3afc19bf007ec455e06c1f56f637ab68f9f711fd85e8d38c5df
Ciphertext 006: c2092641eaeaef13edd94a6731f6db6c25606400b156bba48b8f575daaf4cc9a60fb8226df054b84d1807bfd1ffa466e37f5b3f5aff268475ba7410c5804a7850949efa707
Ciphertext 007: 2d697009c2b9ed075efbbf504e8732eade990caf449627f479459bac15bcbfb54436d534763bf3686b364bbfc69317be42dc45e1d165635f0b64cfcf360edaf1e1d017ef60
Ciphertext 008: 318cfc75c9b4ff87488fbd6e66344a3f0c3875eb828ed5d6e4a5b2177a913967c2d5cfbc1265db7660e2377fc118e353b13badd790cc117fa9c02179bccedb575b528b87c3
Ciphertext 009: 56749c2fbd06010e8adbf970e259baeec86e1e0c92edcc8ebbf745778b2dc0312f0e00b571337640b5249084161edef0d66701bf97d4f517fe46b9a19a4e26b0dfa3e7b413
Ciphertext 010: 7508b5b2849fd9a03662128729a5030804b5e31abd32e5f0c796208a408612be837a26c4466b06019913d32b79bcff1e5d2ebef01db1b4e8153e2ec87735f5b01e4e02761e
Ciphertext 011: cb094014dff5cc7ba454613c4012b8f324fe223d1d752c989ce43a8c6f3089466d92c3f62fb540b7a6f7cb216b2254ad8c04c9aa7084ad7bdc8548f1df9da37f43fd9168c9
Ciphertext 012: a9bd2e4ac681eabf16878c4de92afcd12fc5e787e4e1acc6a9cfc7da06df71a7a9e968c76ca18dbe13f22f60243463c185a061c15febb324105ef79d276b967280e57b5fb3
Ciphertext 013: f397be59e146e6f12faf6cf9d117bbd3306b0750d2ad789c37c7deeaeb57814a75731ca330fcbb282ff59af81b7f3e78095b36b4b8c393f248e3759f79c55a9aee487b2081
Ciphertext 014: 014c881deb097e494645dfcc8dba3044e3e42c1a87d34c3bab5d56568ddd95873cec666da8010693a0f81647719eab6240d53108e9a2a092574b702a386c2116146056f76a
Ciphertext 015: 483fd54af7e9941d895a53a433ca8b072025a7abd503a8b5c3d7c679cc134a8ecf3f75869f10b94ee1c926f5093c3ae4904f804f9bab318a26b227a8975a1cef0f468975e8
Ciphertext 016: 7c45c70460cd766733070cae7cfcbc4122dc12e3c651f2fcf2a43972f4297ac43c8fd10115cd181470a2520b2f77add188c5f8998804221ea03a6d58c2c26ad90238f7ab36
Ciphertext 017: 0279569aaa3cf42d89502783c42cc0d48025baa4e524036449c5c66dbc8e3ebbbf2fa39094c7cb84097951f890d443427f5b2da414c1a5e6cdb6d5ee4880fbb6675694e9e0
Ciphertext 018: 1c72879277c8bf99bff2aed6bed7814c01e4cafe4ff8a31f739bd4501e2114feee0dab97633bf7bc62369df1077c8d31c024e83bc72ca20edc72fa3297a2dc5a2b1a6ad646
Ciphertext 019: fd78e26f8ee7a518f0cac78ce956abcf6761457fd4b8113c0b960ed815a9da2335e6be3fc91184f9242cc12c897c5da856a88e14b0bd14717a84565479e09758b0b90f0756
Ciphertext 020: 65d16c9df2ca41381dff73151f345397f0b72c4678077de0b4163be2f8d7f0e7c90a59d99320db1a57bc82278344d7f1836a5924f675d585c67e5edaed5867abaf21d22b1e
Ciphertext 021: bf412c6e15c5cc7b6e97bb2492f185d261b73c43dbd81e55c23e235eeee649acda8760f165e83d5e2015a504897834a4521467833542f8dd20f3de65d53392b81936f5e2e8
Ciphertext 022: d310def6d9f39dc70d939579d3a0e7cd17d34b30d512e3780ccbb0f0abba5bec4b769096a91c45a94b4b58e16816f727bdaf518495ab08d10041523242fe79ee6f3a60a43e
Ciphertext 023: 8ffc7bd1a0389b6ae6ea8ea1ef80f5ff530dd244b5a2e74b49ac95240c45f40a81fa271590c47a87e0f1cc73cf09e58475d509005cc355e5268b298edd2eec390f04d5ffe6
Ciphertext 024: f4dc5817752ce0a3c60be087f668bb81c413fdc7c14c30d26c57b9116257c7832ba94172fa058d471f6013009d92de30243b9432d6cc0e827dd1580629807a119145ef4af5
Ciphertext 025: a09c920ba92a6afcb982b3399119f12f1b728e3023b138caa0d7baeee247203d7f34c0d8666e413db8ec18cbc0fe0ae046915ceb1af9ca945fa48821d8a8decbc1b0566944
Ciphertext 026: 44f9651331632c8ca957bc25dd3b7be9cb90755844ea2072e8e6f9bd9acafcc35aa92abf7d07989e09bb3cd4e2a73ee3c8de0e5029777991ab132c51cf4737c82a187f3dcc
Ciphertext 027: edbd921f79ce86c389b09f9a42589df9f7da0b5fe6dbcf72b1c6453f952ad6424acaf3365eef6e96fd9b1692370b09eac0fd52209daa9bb60980112d56a4ad20c512ecf52a
Ciphertext 028: 932b2e1f7b760a51c7216673163497ff8d0d1f7a0b51f895e67789969a13f55f4c6cfc54df67d828ac357345941b1ff368d5f5256e7c170abdc61f557624a7275809ad0929
Ciphertext 029: 0470033e4c126d55e1bf8b3f817c2dbeb45c8ff2883c0b907ca30b6758e071781d2990b5b77f0fafdef637b66c5921f84971a17f6f924713d850c7a17683ba1c68fc4ec102
Ciphertext 030: 652342909bd42a4f2dd5f16d5fd5cec35a668eed63cba712c83246a6e6c53aabd092b09ba67066f24148733b20c179ab93e5d66c0def72c39cad579e365cbff8dee37c9a5e
Ciphertext 031: 4565bfeb0a75c8537bf2f777efe9cea8e6b7b94084d926a708ce91ab8beeffc1f0b95790926022feb3fb5106fd060eaf71ebc2a347f0ad0b2ecf3911afe51934c74b9843b3
Ciphertext 032: 13d2bdfaeac9c7d1c768bed100c8338f081d7c736df460bd6df1fbaa3d467aa0401b3ef2e1f16dadebe31e3982bb19585d02cf4b5af52ceb30704e28c934c03d1efbd90000
Ciphertext 033: 0e6ba39beb510471b1d1d3b7dacca756736d695e14e61a068193785230e23555fe4c3c63ce98f1d9ac2a331970b327cbe039f8bd398946da2c2ddb841939e6c6449373abf7
Ciphertext 034: 0e47dc3d280ca744c51262113baf3d97174653bdd84e35268d37050c4f849e6c60c7762eeaf3814f2224e2abb7331511654e6ca93e2c81ba433de18a565917c8e6f4befa75
Ciphertext 035: 211ec91da82c6acc3684b24f385526d827855beff04d98e37bd67e59f29d75de1f63fd68badca7a6fa0a515c825df051ed2a57286fb75190b597f67d81469476486b523e88
Ciphertext 036: 135165b6107548bd1c14e363884a8e67cb2cb8d3c9c4635f0d7a5c42959642bf52b1b1a83a3959bdbbc072b519a7206dbcf1b4e314b1406624b562e943e57145c9a6aae1e4
Ciphertext 037: f23066546d80b3dd12c29860244b191752a7b8819f9f060c7b6ac440a4a87c6dfe74129910a95b821707c2b0ba4ee599a60ab15b7b035f57efc843c361ad1c7fdcdabb4307
Ciphertext 038: d4d189fe0289ce55f592ff8fe3f90d2b8f49bac23b0a262a31beb350f593135877ba0cd65e14b354a2617aab80c30baf3944f538caeca2054894fa877702043862b1edf96f
Ciphertext 039: 74012b0fe7874ef58e00d85852076f9c3f136d7e840f42ded48e51c116cc6a2a8b64b029a4795634ed2a24492eceac171d88ee19b38d31eecc36fce7ac7804df479fa31c82
Ciphertext 040: 361472df8c83a18b97fa91da0a4fd63e8cdcd12afd8aa87a9d431816dbc599514af084295296d625c3df9206aec71914828996c95b43e15feebda26bc4ac62da312a2910d4
Ciphertext 041: 720496c70fbb82af637437ce6774ab46d8bc2f4d968a0231bfa723dd3bd343dea8d5422b70a98e29f5affdf3f0bbde0109b359436b794c691cedb241b75d86aaa4715bb930
Ciphertext 042: 033ef904e77de82e2cfa980c9c59b44f5db47edd30c240b70b6e365e9eac8f1471545d85a6b6eca473e2165001399ac7847feb4c91269e26b0d68a3d2623d831db5c66d13e
Ciphertext 043: 43ed80ea14ce3c371a36653e270560d6a1e7a82651f2c2d358b0210a6dbe00869dc992c9ff618e0e145b956aaa1d6299a0112e03f860ae57bbbf646e42f358066df8e43541
Ciphertext 044: 2ed1af9c51e02ee437e9a22921404bf0ce3117ca24cbeb79fbbf0142bbdd23e4085a105208f40cbaf394a5520f9d13302404191993036cb7fc4a61a517361ac3a298efdec3
Ciphertext 045: e7e730c1c479030194544c592c0c091105af02335793d4ab0c90ef6f46979824d3694c1a7434b18a03f96a890a07a089c0096eb642e3c9f13b5191e38453477782db5788e5
Ciphertext 046: 72f44a134cc0b5a8797baa62f6e1ddaed194007cfd2714458eaa601ffef4bb7c70c97cc3e636e418c10e23359b041c076f9b81748afc37f3419a1e05ed2c7e1479bd7c1e50
Ciphertext 047: b00b5eabcc512b2346987cf7a91c620545e9a0cf7a88dea449070908aaa12ca66bee937644d183170a9f4747b25d63f8c060f11e8585d9089c5e6ef1cdf8e17b426fe9dbd2
Ciphertext 048: d63abbffc77909c334246c0b573cab94cc1c8fae0dee35b3b32594a0ea4346ae19943128a5e2600ad01a785dd14a59bf10fd1690e37c53aba9e296bc22617d7b18560b63da
Ciphertext 049: b5ca1b99b76a47b47850c3a08cc87d1361b9b9c08e52b8d06036ecbeb9e4f5723b3b309462ccf6a85c7d380628a589fe788d6d82b66aa543fa3c3e2c47096ceffc0a6e6a70
Ciphertext 050: bb68422835dc2314473a864e325e3e1a09fb02870c6ecc12d0804b6aacf9e3a7deafd09c21a14499bea5aa3cd0b1e0c31f020034d051276d3439a0434fcf812d00f94cd198
Ciphertext 051: 2ad16240f2349f03c5d6a2170dfa6100789797bae276a0b2d960dc2390cc428bb0ab0c09d8b34fd8fe8b62b818efe70c92da81e7fe986ca913e166a60259838c5a6d9fde73
Ciphertext 052: d5f805a919e7687c4ad66262c347e9d94b6c35a8d061e1d0942b218f8b6715aafb2d7d150481c05cd569a52da1197dfc06f166ca7d53c9515ae5f7ed1294c7b484c85c7a1b
Ciphertext 053: 016f172d3b6639b1a7f9eaf056a2e2430aecd4260df560743262b2066f65ef5c33a6cd190c7275dc2aefbf07edec1d5f8703182f5ff506c445ab358e920f7eaefbc67afaa8
Ciphertext 054: 3b7d0af91e31d2ca326e9ca87cccb15884fa259b8378d5fa0f2c51b4ad48d460e67885b02ad0ca7aa623b80e6335d09645747bd2c3179c6bee85795667d6c42fee73a211ae
Ciphertext 055: 3af1c1919421af180b57411db060eb951f5b92938101ed48780f02bfa586e1ffb86bc68d59e83b3455e6f3ac98fc2bf35779755c8735bd3805df3705b5f0f8cf1c95fdb28d
Ciphertext 056: 1b4966ba31260142ede2dc3655726741b9fbdbaffa049722c95a4e149f36d0ea1bc1d1bc807d8a763401210179c24a37cd4ae7aa2cf5f2e0f673be6a1766f687cf86ba4ffc
Ciphertext 057: d80e10a84bbf12eacbed2caab017ad2524eecfa384689cd28daf2a79b7ddffddbe0108150395eab8d81ce847c48e328331ec555a9503f62adceca4a3844a1a41d149d9f56f
Ciphertext 058: 6aa60272233519e49750dca7bdada11347f55e88dd109a0bd20e94523c657c78c72d13f2bc2bbc969bfb8ccbbb834684cbf44a71a1be8d90491edd2ecb03b7268f9a8db25c
Ciphertext 059: 0e465938552fd194cf18fe187ce7f5094e1eff5fbbb868ca36f42050b88a8cc0ec1d6057179ff08e7c7770dfc87077cf35046b8e91352e91a02dd21e002b9366387586ca28
Ciphertext 060: b7df9ea2938d758481a879b7e30ecafa21bb0b4de4393312bdd40eab2482be9c0ecbf446c339a995f876a5dff30231fb647633389b41aae15301f238624054f707d02f1c67
Ciphertext 061: 1b96cae6e4d74918f8e01cf82cbe4a3b22b142fa3d888afd36c224b603a0cbd8c62e8c18acbd3246ae4c0898ed595ad2dca86049fa168bd7fb3d2ca26d5691cc4fc2811b6d
Ciphertext 062: a996abc7281c4dd72ee0e571bc15161153359656bf63173b756265a794cbfa5c02f6dea94b4599b93ac86474c2eb31810864450b120b05425ebfdaccc8d501049df735db80
Ciphertext 063: b072ede13ad893189644e7fb21f02d9cb63d2fccea45ca1b74dcd93fa5f3aa9295a39972eba30217170eeae459a78680045818c06da16150be66abfa1fa1dc8d7582b1cdc1
Ciphertext 064: fe9e5f4d1e486dcd5ff5ca4475c8419dd012ef4ef844d39fceafa60ae11ef15dbb641759d1b0267495122e61750aa3523ab64eb22af95655d86f8f04deddc21abefb4068a2
Ciphertext 065: 88576306dca27508df5e7899664b05fcd32eff42dc33ee6cf7ffb1c82b1414ae9cfdd3653696cf911c18db27621b2abbadff092d0938395c440cb7fbfe7af5b03970acafc0
Ciphertext 066: b124cbcb5b5349a250cc194985c2e2dd71a7d2f07c2489f035b29d121867b54c1fd2e3c7c32ca69d6490aa417e9233d7d005a56decc4c074c5065412599be4f7a2acf1f91d
Ciphertext 067: 1b1f2c142319dab3fb210fddc1580bb2f00b15cf9aac5b2cd21a2fc8e4fffe16bc21fbad556db2031b403a2c796014e798904c9cadd6b7d4ae7d4a738e193f7c8830eb7d0b
Ciphertext 068: 5345c737d9014fc81d9b7bc4fdc557e7369faf78cb077930944f3302fe98b5cb5a342d6f69a97b3e2fb69b04a90a4d3305bb7bb431448498e0199122467d990e6da918abf5
Ciphertext 069: a53d401a1955d1bf9a3d03bae23a09ed90bb1fd6b878d53ef49241443c911d2bed69f4dc2e1b6ffeb6f08c3990381dbe8340931f03d38db54d9c473f533a6e5364be4886d9
Ciphertext 070: b8f203234c440d28cc4f7e637d5a62e958845020b0d56b270b0b51d1d44d8056ddcf85899465c9f085b7e688dca3c3712ca5e1461f7da0631d8063de7e2d278193c4b85a37
Ciphertext 071: 568cb1caa941f8dd338429e27998d2a58f516d930cf76824f65f0d75a93487f88af0a2e099be4e6d3f7cd16af34e937cdb3a55b53f306c57b52832bee2070fbee0149b91cf
Ciphertext 072: f8defdecd259942248282bf15fc03ceae6834223b84a63246c311ce62e9b8973c71c038002d93d7646079976c7f7d17b22c09ce6b46332b12d786a46732c7e8ba297ab0ad0
Ciphertext 073: cd8024a6a302fff1b91fa0d4c99c70cf8a7194a4cb906c8ea29b3775ce1684f6c48cb7f9ef1023d08c94ebd8503ae2bfc8e2c8eb3e11e86514d23ab9be6863d0b0f5d9aa44
Ciphertext 074: a743f55c7abb6c90436b2dfaee4d67b7a9ec9a3fc0b0b4a1a8d05c0777776112f7ae3f8dac2ac923478fe4a8b4fa945131502d9550b858a682abbef3eb0be5a218cc3bca35
Ciphertext 075: d0006dfef393c8b5e285fbd0e1a5360ccc072661df50167b09c109b8bd0c581e70fcc35a57322d314e9bf98f737a689f5bd51f21a7a38a0e97f22d83cab505799b337d554e
Ciphertext 076: 19373467b2e5f6ad875bb2c8a6229b9542050829687ec7565b3db87a317bd39fd960ef5afa327983650e884291cfa9b607828018ac34b996732315599550ce280986bfd7ae
Ciphertext 077: 91e46f185e6dacc9f55f8e8a35caf38304667badad4d0c09819531e1dea7e07fb9ee14d39fd5d6dbb88e4ddec64b1c8b4636d39e0d6f907d45c2f41903e2fd56ccad3e57a8
Ciphertext 078: db90654051f1fc6283918922276837c1df239a67f98638ca5149ae00efbaf2058e53d3b25fbf4d114a8459bdb7b5df43304376fc398f116bc29f9a9de5f2ef9f8d6fa50571
Ciphertext 079: 61248525146982f93895b32138b8c7e6f2a030078c64fec08403422aad4358489aa3fd657a2d6c6837f54d52c1a282dd06e4a06b0ab548cf1fe38f0f96c0d8e36f44453402
Ciphertext 080: 0805834d6bddd82fcf7e0198f6392d532a56996591dd677084e25ed6bd467006c156b95b7fa0f142904f452d91a9d177c47c617b74ab5456cf2e6a7e103b3ed9a3ade92ea6
Ciphertext 081: 801fff5b6c5923f92ae22e9a42d65e0f871a1b49d20cd1991be1d55150d0d1140dbc31ef9e8d4571e1d30278c5d5e33c5c54ba779c3aeb95db20c2e315a1e14147506ed88e
Ciphertext 082: d28051a8610e2b25e695ef212f106842a98ce79810ff77de0318afbed38f9dd0d80bafa1f47af8ab5441f8ad9b77130e61a4011d019f6e060b44b406b210586bb7a0df6071
Ciphertext 083: 86ac1736bc5866a567952f7ea5fbbbf46a853b62c6dbb68ff316ff4cb72e25deff1bd40de6c0f93d7adf5b8e100b875b3ad9d690959f43cb3528a44cdea344ad062c31420e
Ciphertext 084: 7f561902505507a23b92b4e5a23a67c89cfbfc64f75776cf58f9b19830d8bb3fa3f6e0399bd794cdd0a2c4471fe76184058a66f9f10fe00cb3b1aaf8a9a070bbd59f701d27
Ciphertext 085: ebe31ce1061246be8cfcc60c110a30d49226ec547fba6c9e2e038b76bf10bfa43419c193cb6400c4d47c7bf24d533e55930d51a54726a2b75f508821846fd93ef7004d7f9c
Ciphertext 086: c0e3b2f7e3ee4248d8da1ddbb3db40ae0d1176333d8ce4c97b6a9b2b82f4be9bf61c7e1d0a74a937e6e9f90c12f0927a86676d81c95320d6dcdef94352837002dd09b8dc48
Ciphertext 087: 6b6665e19b4e8fd50e8c8f27bdcb5d58693e023b8afee717f04c4f7c7a036acf75bfb4ff29d674456479f16657db164658634bd1a3f9bf9419fac2f69a8f9271519b426aa0
Ciphertext 088: 6cbc9f724e4bf34a73f09757f73bf24cfc44c142c2ccff80b89cd170600491a6f1a4f5939aa2ccc6f1b0ca23a3c74e93bc09f61939f8c946403f44bcce49f6d36b91779d4f
Ciphertext 089: 5a53be38064e05b2a231d2f39d5b36e67e2356debbe903b9e56f9ddadf226f7d1551dad0af68ad2e44c1eb341541c27202a3b847aa0c0c013729be3a5de1013d824999b685
Ciphertext 090: cf9bec1356bc040ca7555413c03079b4bc60a2d211a078408b1d1c1a9d9d71dc5186eb451d11a0a5bdd90a7785ae2c68f05565c8dbfe8ad7cbf462b5a03eb873c11c4a11ae
Ciphertext 091: 68c1a335a08ef2ba8d71ce8d714d92b788009d305e49be4b89fb01792558b2f196e3e091c96f9b8212916a843569ec9c48c606d78a434a552bc29688ce9ac4e4aa8627bccc
Ciphertext 092: 0af5648ad53a9a2a55404c1b4be96750fef978167f4215b05b40af1fe3af90250651799ecf61d2dcf5f837c32558f3e0d839c6851b02e283733ce6824d527f75a91ba38ec1
Ciphertext 093: 403802469f02d90a397497aeced59b92144d0d19e902d02877e2d7e2368deaf8752486a558efe85ae047ea8af2aced0af131ba80a7c44d1320b6c04bb95389a0f2c1aaa952
Ciphertext 094: ecca27aaf02312937014485a6180910354ffb39c7ac93d758e41d54035b512b96a4529c83068f3ab75e336d8d1eacf4567e1c52b3e7628072a1add2dd6d56c4fb41c445f55
Ciphertext 095: 59657fe6dd9b99b3316a5ab625d306df2897e486e2a61c0ae47540c1983bd26bf8e7d867473660812067f6788134af4b83e4596e2c8c25fd32631ed890b123a56ad019943f
Ciphertext 096: 482ef59fd63116d8549eece62e6a579a8a4f3b0e52d9f9ab444d0cc1adbfe579b8280079c9eb1d4b69cd1104dcc7f35567fa099bd6f5039443bf56f077870cacd751c4a790
Ciphertext 097: 91a7f8292947574430fd5cb58f355134be7eb45dd43100c2deca7b2167afb4888bf6109cad442b5eeacae26c63153b0cdd260cd6667b12a62dbd3b2cc2d6071f98b7eb42e6
Ciphertext 098: 18b9f51a985dadc678db564384e0999e1ab51795d7b69ac345d78cfcf16f09f1c27a6964637e1c5a5e6dbc6748a6e77ead98e622950a7ea1cc73fef2ce965c21ebdcdf3a05
Ciphertext 099: 794d24a08f07f472477c4ae964e0351caaef06f0e3fcc84847471e579015dab39b5832e454463b3aa5e06169c2ec533c1686542b18c0f44cda63ab87b5d95a2f95e156a299
Ciphertext 100: 992685404c94e916cccbe44081307f1354c10ffb25de0a990e74b48ce4895cf54c9a8e2901f8a0be2cacdedbc33dfe9856bb03c92822addbda87f75c1027afe6fa5e172bab
Ciphertext 101: 5cb4e707373dfcbce46907e99bb9cd3aa8d49f47a69befcc7606f329213b586bfaaa502e475387894d1daef66874d58955bc3b9f21851532c1aad51716897ad887325305a3
Ciphertext 102: ef14374f6430dc8315993b234f177f5d1446f152fdfc5cc824be8125667e8243db17c017686a7ae2ec79358007dfb5dd30c9f7d99d8405541e152e184fa198e93147276a9a
Ciphertext 103: 2fe8653d52230db32d1e36983bcb628dabda5dfebe5bc1ac623b16bd8f4d33abae9c42a26646273d26c37e36649343cb644f5ec8493a6569dbd35940def4e338cb9a53f234
Ciphertext 104: e1d64305ee836d0bc80dfce0489403cd65c93ac007bb4390ff2de2eda961acee22f19828bac94e6866cb4d2d3db95518a3399cdb4d6b33d34afdb1197ceabda61ebfcca2ad
Ciphertext 105: 12eb4e2c3c6041ad1ffa5bc62c3feec4ad2279981431175929642d12f4e844b613d6f0238fae589f6a0ed3a694c3b36d47b478f150f8c8ecd0e6f2f2667ac054c972c05353
Ciphertext 106: ad50bbf3b05aacb9db05a53e70c96258de5408f7fed774d9c6eb39af1c6fdec733e0ae761e070c5a0c5498828a817e76fc10d80f88c9363153db9d19309d89f211d5a16014
Ciphertext 107: d37b6466764a77b0aec39f04383817d01a9911f0e0c8833816aedf7b21371a27f0d87f58ee720d58819d2a9198f1af4e0e6ad520c1f88415c6c4da488cb846dec68c8c1017
Ciphertext 108: 4b35e13033ec05e63f409453f925f94025d2e75047a10934ff91757edb2aa77a55a16a98ad4658d8b7d6df8b2e83c4a4ae16da3061a74854d9f6dd630b389e283a13e71fb9
Ciphertext 109: 63892e358ee2c8aef8732de3a607928cf275282cf7430da7a2685ee2ce69054f5ed02577479f2c6cbf21ffc04318d214af12e99a2117885007e99cc2d34cace5fc2c301b50
Ciphertext 110: 2c940d06b9195a65647adbf279e4f36baec1b783599115cb6c3dba31283da795bffc90ad155d1050761d623ec0db39e6bd0b62a038ae1952e4d13f0ca9356a44ede3a51c8a
Ciphertext 111: 6e3a9aed48215a496ca73427296e16bba0745360547fd7acde1bb382e7a143b76beed281712a8dec849a28edbb04ae48ea4297f74c2641f2aae31c259a771caa0c19681f55
Ciphertext 112: ca2f00c69fc9bcbb35f70d025e6a7ad9b1f339e0bda543a610296d0f3e1d12bc4d7a9e22fd5dcf4e2ad9b4f58c000f6585d7cb353006a3286adf87eb2ffbabddceb192c986
Ciphertext 113: f8b7d18b513703fe36e711172b2b449e39b6f57f66e8ea7a0bf4bc9c98bb05569b87366af6d7d86e37e29bd2cc28608624492a3b2cd8f9f5dcf89bc7a6974bf7113b1ca178
Ciphertext 114: 61a47d2fa93020c19079a38d313d13b48aca8a474979e1b6b3f5635ebb806dee11cf969dae4f85ba030c584d36adac3a17585db7b93e3f29cf13669e60217e581a20a03aa2
Ciphertext 115: 0613640d31eae4b95e09e21347e5236ac4f05d41b8700ede78f07ce9d927d54068c284f61ea8195b9b4fc4e5f75b7a5d8c40a4b19961b657d453c568574f4820a10626a956
Ciphertext 116: 23bc2abaa1b8414d2f83469003b3f6d4b039c104e9bebdb36707204123997172b546497314c599034ace37bef868d6e9e9c4faff373ee992ba8bfc40f3f0fe542856b1c7ac
Ciphertext 117: 17d364ca72769a8f51a33f09589226f61c24e6f133d5f587161e9a1f80d565b76beff528f2b0e2a53d0c820211d8eeb68d1553a4d1f662c0ae0ad11d0036f4722733a62003
Ciphertext 118: 328dbbc0148a6447e72c181c77755ad901035238d3a05c9f8cc503bf60a36ab0786672a1a0ec1d2b4535827d5b06c0fb0d0db22c0663698d1a79d22e492b8b05c11c76b25e
Ciphertext 119: 56dfc1e6e2da977179a302c32c9d164f4bda9428f5fce3bcf554ebcee0040bf16fcb5ea576859e94bd2d063b67386e29eccd959620cd5a562ef2181b3184d73ea6bc6eaac1
Ciphertext 120: 9b95b7a33d46abdfd770371c11628e8d5d659801e19f0065141813c269d113e5451c81228b13740794411c0337ad8979838f2305884788b81b9c9fce262411977ad5483449
Ciphertext 121: fcdb8ed0151a6d8b5537970cc8ec7129a60c9d4cc2f32444a75f59c7127bf0946bcf517325e570718dd63e743d1e35c6e93ee5b2f66d180e56bb6635d47f331f34b30ba181
Ciphertext 122: 295a1ca9371613e92a73843a0ab7b4d6da86274396bd049c80eb5e9fa1a6b47295a76921f6524a135a4e3265f038116199d78a207606dcdec6f09f2070b27614d4323616a2
Ciphertext 123: 3218bc8f78da5ae450a228b4afe75f467a1a07647fa146b3c61fe471a97c74fe934a74f6b61c3db32471382cbfa5fad623d17814f18779da3902964b4196f2be31b09629e9
Ciphertext 124: c63fa3e4385b3001cc52879a3d91db8dc1dd9ff0c5e2b5f70320643acfcd4d738f85181f8c528458c28268daabb34ff3392e390f72a2de175810bacc8ac929c1bc274dbc70
Ciphertext 125: 9b2be501aa0f10a953049e0b9214a3f09296e40499034f76947079c239dec210eb1b06f3bf5d69be81b8f989fcd51a4d432f5058a2382f64a61d8a5cdb76be0a1d970b2eaa
Ciphertext 126: ae4cd4e8365df30a880a13e1cd07c0650ad2bb7d9cc18ccdf925248fd29bffa87f00973ed0ef0c982866b57b26f8194e8fc42a12dd1881dcdc9e5a5ae4c8f4efeacfa59570
Ciphertext 127: 93a129c3809c23f4a7c03548e0679f86b206907f3c57f0771b44be26dc623c2d28eda64f1608ce692778779101da63bf2d1ba6fc82612c7bf1a4c9d4436b2c9f7ecb2a704e
Ciphertext 128: 09980fde5e010998a0773abc26d00623e8bfe96a05c0af2e6ccf16c7768b2eb4626fa73990a431a7ef651a185f32ea7103def79bc0f9aa9b36d8fbd88b202f0731545ef1d2
Ciphertext 129: a1945893a097e4ba967c270978bbe579c24452ce928e1611916bc41b7eebf5df03e3d341992c4a0c5b7e4c7125308aa684d17a2194092e571fe18813304d2147340f169f0f
Ciphertext 130: d691979223603268e3daecb20562d0dfd99df7e6a95f08be9b6cab1782b3525a684479996ce663765a1f283b74dafd87bd4e58129059266dc320576088952968e58340f71d
Ciphertext 131: 877fb7ea990be68c2a919b36b2c94fbe7a11b3cde98f0f3d10617887fc63682a1f8784f6bc922e372922e82f9f840d98806424e4bc97436651de5664d2bfafde2cdaaf64d5
Ciphertext 132: 80f842d91b0de6bc758e42272ebfe8466a7531a6f645ac487f572fea9f6e1e9ddc19be0bbb9d5eff47e85701e6fbd21f9f89489837d8e210c8280ee12163cd7dfedd49399b
Ciphertext 133: 9f762946262fea58d012b4cd8042fc3cd3fbe969068570503ee888571847294d010fa8c5c0ae2845ffd6efb52dda8e3f323f8049617e0ed85cdd06ed0bf45eadc18e4f6376
Ciphertext 134: 89b2b42cc7ac9136662cb0420f68ad21ddc68dfe583e83be4095fe728379b8df4f3b15aecd7522dd1abc01946d5b032ca13da8bb96aa8a691bf84c539357f78a8e5b97cefb
Ciphertext 135: a22fab2fe41823864f7d12c13019ce232d94635ee1fcb124b394f9361f86217bf8cebe347645f18c93625be3be9f8802e7ec27c1965541d81482ce883e1c1f795a9f670d4f
Ciphertext 136: 04fa3f13046a8db58a143a4ee76a165a83e229e15cc7289e0eb0330f221336253380022767a8c425d91fa6e6536aea9cf1f4efa02d4ab3ba5b0cd8d5d78ddaebf008631780
Ciphertext 137: 4e2fc5ff32d36fcda2a32ad565768033fd513a850b2e2f6c1eb45dbdd7084d0ca2877c6dcae2495d2e830d415f4f5978889ad4961b3f5a1a969dc8b291398633dfc08f7244
Ciphertext 138: 20e8ab855a99c42e9dec987c3df16892c6ea3f4f7e94b5522c08b746f2ec03e5d6a4a9a7aa6c19af94a0c78e53f330cd691c6a4b000775d215e7eb3dcc63a2e3250db48550
Ciphertext 139: adeb7466d1d0d5c808b49e00ada22c74fcd4e0242d04a9ffd44d540242e4fec3fd47b7802bfffb8c375b0829d6c8def7cf85b8f36e13391d5549747bde48c38e8c296b207e
Ciphertext 140: b5a5cf35c80a2c3e454d30993c959bd3fc22efafe15dc9c2fde698ac7f2154a4b4d0cffd711eb11e5a22f1cba3acf9a4f3455bceef727ec48bb036c61c099639cd5719e844
Ciphertext 141: 4e7d3ea3c9070c21f517a08f27e167c382591090f70f8d97f0e6e8212db63550dc00ef3a4cc98a58126426ed87609afcf2205f364c0e2428fc9989c962044b8f72ff25bd0b
Ciphertext 142: 56c467644178acc4ccc32be2717423513e30f12318df45aea71e2ca4c3c6c4c98ff5dfe8934244082bd1e75f919387e23f0c3e8936fa3d3d37241e44069eaa8fa65ce116a5
Ciphertext 143: 009bfdecf76f78d59d77824300c8da202dfb05db1f14735215f728195f8f1cf7f3e6e52279c6612f8de8ca23b3010434d80277f09a52f7ef17d2840e4242eb1301854473ff
Ciphertext 144: d582d45a3aab9ad201be0b95b63c3c86521ae38b60f303ea78669519369abfa294373a0a44a94dbca8b1628c759fa662ed13c20e85aed85832efc31c7792ed17492587fa76
Ciphertext 145: 5c950c09152d0533caafd8318428f56a6eadd5c522b13cf975b8db216aece41dae0f5059542b6b93698aec388853de8db558674d30e2d52fe25c3dc22016ab3ccd6312213e
Ciphertext 146: e1fdb3b4bf498b0bd4c4a49416fad8ce25aec2c3b6b8910abdafa36360564621543a529afbb5bcd748f638c613f23be114b263546744ee8543322a18a699093f24b9d45403
Ciphertext 147: fa82c4a06da5ebbd52ef9d106f3f1d974e5d67b94aba07869cb9b7b8ad6465285557b41018d0659d4054a5e1f29d61ba4bbcaddea8ec80db288e9e9a8d44c09e6e7f75f9dc
Ciphertext 148: a33e06b07f57277486bca104e8b3b7711a12dc5ff3eb8ff744671949fbb7fdf3953cca87baf502425a9b79f3d2b98c98a21296bc0062f38957f1566a6b4ff2966e920879c6
Ciphertext 149: 78c90646f8a0660044dfc0eb897c3740a6db651b247c00bf338545d223f37b124b54b73dd8849f1bf7e0c7605d50a31a8978a90d4a233394c09d4e684ef0356c3cff5bdbd5
Ciphertext 150: a5e1f4c705b6e5a49abbbbd4ec023b5a42f5c34e995120ff4fbd9a8a679b065351f7ab3dbb2c416e17c5bc084e4fabcc78c68e7535c13bbec7e39b90d55a421b7cce3524ea
Ciphertext 151: 74b40aea7d69e96aae1a4cb4c685e9aa056de74afe0df21f9bcf65bd153ae03496e86b5eb15a33282e8da300e8e8cf3eafc955323cd9b0573c89e7511bcfaea1e67e7fbb70
Ciphertext 152: 6a10de15e3ac68029365947fef4a275ca185613722a240ce0c38b0fabf5b0c0c5e6293c8f46250e9d2bf2580b0d22bd4274aee1f25e5df24fae082f22c22db941709e1a9ac
Ciphertext 153: f7f75aa001a905f4fae7e25e50a1a74f1d56e995d58f6ca59406ea487e76b0fd136a3482a8867398c9ffa42eb04b8f25029aef41e6c0bf9ec77b0232f6c08a787f31a2061a
Ciphertext 154: a9a25d80b5eba7b99be67de5b9bfb8cdf654f46070c7f1b10cf1d1095e2d5615a64688613b9ae132d721dc45ad221e539a717db070e46655c68c7cc09f3294a963ffe3337d
Ciphertext 155: 49d67bd8e58b2a768a78cb2055ff2b694a2a5906e5263ab86dcae39292c1f92c91c325b8b319e36133275a960cbbf28fecb13ed603d2b526cfa597114af51b150bc3275fd6
Ciphertext 156: 3fbe320b9f467dda37d0c5a9c74fdfeda433b8e74b2100189d75b61c7de5e7f48b3cb011c652b9454cfa0964ee612ef4dc9afcb1fd361bb654867169c70b54227cfe61edc7
Ciphertext 157: 2381254c038e44053a42c6d372d92c0df07ee767a6e7eb5679163c3464eeb5f11fde321eedd34d3b0f2500e0a8c5d48b0ca35f1b816780a12189c3d3c91632ab2c588660fa
Ciphertext 158: c1a837fb2c22dffd5db416bb063727d659c73f3e5b7f6875efc3aa1ce7dad0a1f31e3cb36ccb045c64de94b45c8e39f8e88f8fc6f17add296c8434fc52e3979150ad987678
Ciphertext 159: 39738bd33094669fe5b179356854db4d3968b8a067b7d884e08a1457e05a1b1020453e2905353ff813febc3cc283cf85d1ac78bc14352b5285cc45cba9b4a356c767094142
Ciphertext 160: 39fa113b5d05d815a009b78249b687448d5626d6d1191414ca3dfc1c4c17673a0ba3ff8ac5d2171640c2a076c631a83b0e90456c75b93d9aee16328b483b518acef579d7eb
Ciphertext 161: b82fc9c3b9885a5cbf9469ddf2f2674441bc710ae637954f33fe3acfd625bc829865d3d049e00804e191d07e326612971130352546a7e4ecb920a1361795a80a31607a5033
Ciphertext 162: 7896b61aec9a52935e9ecdbe12eb0fc145435830bb1835b442b26cd6e757cabfd3cdae9e11dfae0342c28ed4c47161a62eba61899b3d0b9f9ab66940e65e016953f49f0bfc
Ciphertext 163: f39e7bb0475d9487bf565e817e4aa8569151392f87a34e2640b3e0079c37b08b5ca0fb50ae1888642fa564eebb107b0644c91a06a0f9c04a3272b50d0c739653bce0308bce
Ciphertext 164: 3cc64a4a99059975acbfa92e221405ce33eac7501044561178b67abbec64e096107565589085cc60a81fd9eefc47686a39b1caf0fd575a4793fdda971405b15f565e738e39
Ciphertext 165: 5aab6c47f5ecfbc849e80ee4cbd6e2635c96e86a974eb7a2334436ce71f30757990ed6f989290ccd3d3f40a6d4e02cd79d95269129f111a3fb6bef3a62ce8fcfe7087806ad
Ciphertext 166: 8b14dd2989a72bb8636db72a75853ecf9a218db535136cbea1768d0af71f02a31910789d78ffd9063e1299e77c6cb716542cbe4a3b45c05810c84a9d7e31b28638ab7c4562
Ciphertext 167: e8a48f50b59e0e3286ede1bdfc5e62e2cfee95bcab64d4428ff43b41952387b776d16089569cea315f46cc347b0f84824cb982079661e3aebb4979d6305467c80ce716ea65
Ciphertext 168: 8ea0dac2e3d119831b8d68e8e58921dfdbf2af33f27c81c9488f5c8dd3bed36d0f911b22279320e767743bd6139144f7ea109409bd045a35a82a126199abc30b28057a2d85
Ciphertext 169: f5d054faeb613cf6efb51cf55b8ffa946fc4cf9b42af26660211b4f80e69de9e829ef6e54c624c4c132a2eb3a5d182be694c05c613041f166454f5e4e6d090b201b1142e4e
Ciphertext 170: cd01005c3f50405c03011df75b034ef77d73f3937397defbb6abe7da12226325b377a144d6db50c4c3fd6397bb1d7de698429f8888fb6f5e8e4e788b003fb2b8bc2f06814f
Ciphertext 171: 828b7719e276ee1045b9d02e9952633a80a23ae7d16d77cdafba11a24aa7b96d860375c218c34684e855f481f521e1e0808a45a12fc84f477ac85cd4a00ccc03304cc94e33
Ciphertext 172: 1e60bf36e778864a6a485de2f299d73c51e4e74dd8cd2896ac56f0208f73550e3aa58ab92dd74ba1f0271c070bc43e56072cd2fc2f42eeafffd37bb01e8be440058cb1a8e6
Ciphertext 173: 0dd100cc47e24f2997b655379df147ed098060c2c3c8fabffb62f41387efa1100fae646cac70312b34210f52921c6aa66359516a5d68f6659c3e52f06a11a7d68f24fdf262
Ciphertext 174: c830cdeece5dd085ffc5bb4e1645b5d4a99d053d95bdd42f623cfa7e8bbd238d8243e2e817e7c10441a0d778f06c5173b2c142b6f17b4a0b2722a237a6ea80bb798dce44d1
Ciphertext 175: c7e15315aff54ee5183fc0d50aba6682c64a4b98b7514adc9fedd53c0ab333986146710b89cf0dee2e00ef865f3935e4c8ae51683341f741f1b097a2c4848ccc0aa039c4c3
Ciphertext 176: 1542f404ebf84a631cf3c6ff102fdfd4f6c74b37aaff2869fdfb2a3f7a72d83cf9de2263f455cc633e177428234831d4d555d507865e568b7c9444ab6542d42f7ff7f36687
Ciphertext 177: 2ea5324af1a9b12696df39a3d98290f2e278de990e97770f348f0227356be5793ed446113450eba40566f725325aac8909840a235742497769267a3cd38e9421fb95cb357e
Ciphertext 178: 4c9d8f6e140e7dd3adbd1eb2e9fc4f10bbea44a86d05799497aa408d861cd95af05d91cfcc99e616561b8480b14e50b5f0f713442be769e72f750abcb945995df967007eb0
Ciphertext 179: 31aaf5806a0ae3d341fdc73e3a6317377550c5f20dcadde239432bc9dab2f07575a86a15fc0817abfd02e96c3a92b8efda1c3fc5ba0b9fc260492a5fd25630728727f989d2
Ciphertext 180: 25b11742edf2397b4871fbf90e3d164e41db7c170186a825050c523c1afef01fb443fb52e29345c77bf7c43bfd806488d2cf053015f9568032f9d929e376db3d06adea9b2d
Ciphertext 181: fb1ffb60f7287bdc62510019b9e6360319c2d3c964062d5be5a40dfb750a92a339fda1ccd97654fef20750b37fb9e27ba1101f42d99422828d7db697e80df84a466863146e
Ciphertext 182: 04d99e9105296fb735eafeccf6259367c0f0daa7640873331f044985cfcc8a0eb86780380b2ad50fe9d7392eb55f2cf9c0de3e412cfc2b08e44dc5ffb32618a78ec7a6d48e
Ciphertext 183: 8d5fb64ac91f75bb5cc9ea373f5de3ecb0cfa0a70a7e1a78ecdefcbcd509a77501273454dec26841325c6a8d1d179d8c21cde53bb0d28b485ed08105bdfc62e2d65f4fb790
Ciphertext 184: 34d58e5f0a627ba2b0b67bbf18b186cceece4d68f37d4f2032b201928f16fb2968fda33b81005883183d295848117aa1080256deca9865cc6d81a29267413c62cadc7bd2db
Ciphertext 185: 0a5d346472c954def8fe6e6b6ef3e339dbace9aa5c7f5d649135df94c5caf71245c0cb373a26b766ac66e8a19f46920a47c701836e450060179ad6ff5fc11f4af1dbfc4eed
Ciphertext 186: b53c9ecd3582cbaeb9fa7231dde64720391ebc2c57408a6463bd198154bee285f386657e17f0475d0113c9f97afe365f5772b6f1ba064b34ea2f63c3f7c24733f771e5332b
Ciphertext 187: e9fab21f896ef200fac1a936fe3930f71f0e1cdc174a9818903502a4815941726ead8c96e2d736dec2d21f5bebc035962a9bee2b2a48272c0132e90880b1a8af4801bdbcd4
Ciphertext 188: 39a6d3b5338fb0ab860ab90bc9d8b97305ff9c549b484cf95f7a8f0b4adbd67fab3f797eed095de5ed4b3deaddf158a68a3104e5566bb5bdd3dc0d67b20f782759388b8860
Ciphertext 189: d195621302bde3f32b32e73d59f36313e38cee783d19e3602b169dbb8a0316bfd97c6e6fcbb686935b31faa735de87d1b8cbac190b6c744cc71e0ef5402c9215e8685b3681
Ciphertext 190: aae22251b9b7616456bc078b7b360759c0bfa884fa39e135e92b934ed7d1ace9b90889eed6cc812eeffcc72ba42607ae8b466abbfb8c00b6a6292858f218d6c5ccc4f960fc
Ciphertext 191: 0b7b07b86c3298446c4634970e6d046efc13e648ccce6f7537532208c73ef033ee4711d8c8fafcef34fe6c6335e3a334f2bcabb2786cd8dd202b1982401488168c7e472da7
Ciphertext 192: ca5efe3cbcd9969a46df7e514b9cc3d02dcac4cefbb798157a9f7ccfecc1e1a0f799ee7438799fed6869b6f69d4396a7002cc4e5530eb849c1b59fe3ff1b633c1940e4d633
Ciphertext 193: 03edae2252f7008c051aa0ba0efdffe928c898d77c1cbc3112c4054dfa926e8615e4fd4d62608e539295050d2d728683af023cca6f5f76c40e463ea5017b0414166718b344
Ciphertext 194: 375eb8af6bf3fe4a817261e1fd639c7415ce46c23c26507a085362f273108b864b0dc4eaa312775a5b1b9ee51ac9f5366d0ecc0373f9328edc3269d8ffbb89fb53b96cf5da
Ciphertext 195: 4c0c992210c6c38056347f0a737fcef9a4fd57ee73ed6da5d6b0b02142ddfa93b322023c5359be116247b9c923964daba3311d02af2914ed009453257cb5cfe16b45515ff8
Ciphertext 196: d414e59205c3db917d83cc751e827fc77705a8cbdd2d7d46e334f6f4c11dac120b07f7219060530fbda00e19948ac8830bde05b8f6c21e367ab61d8da7f774b807e1c19bfc
Ciphertext 197: f910305606ede8b5064f511efe8ca59bf7c9998695e67a1df8579ecdb86cb30e961c5c89273bb821029aabf750d5ef2c36178b9673d22cf0049b67fb60cbac7c23cc35dd1c
Ciphertext 198: cd65fffbe62551ab4856da36ce56445847d4a80b64f1ef47e2cf1b5b2519197b8a807df109c28e78fb926edae2cea14d77685a0f20d98c605308967756ddf12d945ea01d4f
Ciphertext 199: 6ce400dca923c3829fd7aa0034f271cf3b4f85c7fd42cef37c38bf2b11e3e344acd492f87fe1f9ffa8f8982527fc3f08e8e55b64bc779e3425eba2fe44eb9346ca11329f1e
Ciphertext 200: c8c9ec5ca8d5e1c4d1d45b81294d7aa5bdcfc460f0d7156014aaeee90cd06590f565364af5a1a4707e48ece1b16c91c4962b1684a5128248b6a6ede240527b4a68d21f819b
Ciphertext 201: e84c57743db2a90fdc53515ee0ce81f6801d36db8ab0a4c7a426a3979231055fb61e0e1808e0b3b794e5ba6e3c1508a402ccaa1fe7962aaa219046f98a3516a6dcbe72d34f
Ciphertext 202: 399d4e22d83563600d2d64509ca28c2ce09e33e66226f91c3f44f6e1ac113ead8189377e5dff2206ff3f2d026a004a32a8d466c744e5a7583de9ca66a2cbe74db6d38c438e
Ciphertext 203: 9d5f2603b9706150506ef6da1eebcb1919b023b807d0cd46831f228ed535097e0f79fbcb1cd670b31f21cfed6ffd8a355d447aa847e931deb45b635fe5ad62a97009b77d81
Ciphertext 204: 19fcba0bb0fc53553a991bc9cb949a4fe4f8d1ae4a84b00c2baea147b9e66884779e5fa8c505a4b970023e8bccf17ad1cfb2526e8d63542372d6d9f4506afbf06df8856793
Ciphertext 205: e183ed57eec8f780416e3460e25663a1c3274f043fe585cd493a50f45482d09c0f10483a5ba5fc2d72c515004333b2fb55f5eaacd0ee6af1605e2e0127ef7c44794d1426cd
Ciphertext 206: 92ac84b735e8755709129e706eca54051f14bdb7939e03ecbcd86817cdc8e474d7fdb00c016b830afc6992999440855952c282a5fc0bb3c84f697560ae646db4c8252923bb
Ciphertext 207: 893a87e5c7ada2d297fb17792541ee52f1ccd74cb654770d4b9e2f7e20282e2cabb3ea30d0f6d9bb9757ad2526b2cdcec8cdd11874eced13c5a5c6e45ddbffa4628c44b7c3
Ciphertext 208: 613d5bc859ef2257ab35f77eb69d6ead9aebd715f2bc5e1e9aa0b9a59520e68035a6605140ad31b4125de55cbe2849c23f4b8a14ae011976f0ba9b714f961aab21dbebf40c
Ciphertext 209: 627677ce87fa3f33b955cd52e21eb2e0037810e161700f574dbbe33cc10c3858a7038b795aafe4fbf669181bebea71f7a082199009d99a00ddb6eb45ed2d8b4051e933568d
Ciphertext 210: 623b7dc8a9fa30d43819c7d713c429e55151c510aec914b11a07d628e55921e4c1a0e9dc42eb20e4280c3bbbc7541ed2cb3b3c4316b91f7c9c12e0ece8292c7d6b59772f12
Ciphertext 211: e6da2ac606210cfb188a89147ced4c6a96adc25d18f8cae43e1d043ef21c0861aaf9f9456ba6d4a5b2ee059c8ca184c013a1f327c98831e934224a1592efa8a4021646934f
Ciphertext 212: bcf08bd796373543e9e2bb828a649494acff2842f5f19ad76822b6eec3b0e164771e825c9ad2a35c54a36c936027f7b7a50c1c9a09660e7e3d695601facafa143c2e35bde0
Ciphertext 213: 4ab646c76c1f413d141b6116fcb6a446d826838ac48761599be2328e888c4b699c3cf7e3e23a17b13dc414ac102f6b86aaa82520032a88dd4d9ca563d5733156533f02e39a
Ciphertext 214: 31b6c8f470b4e112da292a362c7990f24cdc5e7d319515245912c617dda6b64d539ab70bd39bf05881f50030cfc05389b871fad99d11364a64062966d5b4e23179700679fc
Ciphertext 215: 960ed4c5c256f64d4747392448203f5aa3e4ca428bb34995f72430d969fced0d0f2b79c6ae9a92cfbe747743a9cee9e296f243ab691ffdf1eed457b09bf3a781eb8a2888a9
Ciphertext 216: d0e68f21ec6819788c0c828b7842a7e84d42486b9a5e535d57c85dfecaf2f4ea5dcb44e9a98dfac8e2aac241f6374f7003aed7fc7bfa90d0f93ef9cbd6bb81cf06aacd816f
Ciphertext 217: 03d142013d602f0e55fd725321c607dfc71b7b01e4fe2fd52c84a7a57b7c23f39fa3dccb6f20c5e64caa34f0e50f4b68a030bd461e3e738660fa4715ab9b8c84650ac14c53
Ciphertext 218: 25474293736e25303f8e4c9be0346fdd692951339f193c60c021b15b57a6317c42f209185bb2f8afe4c6a986916655a73e02482a4ec2825c0e790e0bfb0f9a290c46f205e1
Ciphertext 219: 1f2d9b4f4596a8628db2079316ac16cab29b3a61b165f19cf2a9e3915820e740c4a55a568455aaa1dd05f5707b41551282f6e127584d47b8210a070a3efc7ee576bfbaef1a
Ciphertext 220: f9f59b517d5076bd1bfb74cd298a15cb0f757f95bc70c31d0f92d8646733c38e67e902355c4cc456776a8c1bf73f6a523a717d7a43daf0adc0a2a1fdbcee089fa09c21cf66
Ciphertext 221: cc94289d4573d6ed833b76d3505c33046949394fbce894a6ea5f18728b651e7b564eaf6c14a72fcee5464b4dd85b719ad06dc15fe1d61ac5bc0232d16986871606e80d6556
Ciphertext 222: 13f8bb54ef0171ef4b1eb207015cf3a9c72d770e8ef45705da1c8810cbff912afc90ff9c8b92cbf8d56d47c1046b44d4cc6d67e4924906b52a1e93752f870b77a633f27436
Ciphertext 223: e3795afbb592b12136be8257322c6ce57c3e28b21a7494f5ff09b846f0e0138aace05f84c47f7a9e5d028adf8a752616563e4659b5888f5686b58632e82475c125b1154363
Ciphertext 224: 60b3f44bf0f0b3e2d015c83685048e0ace96d4d8212b260644c3e93447435ed871c4f7861f6ebcc115fa35b332c3eb4b8de020df2b1d4b28fad3470ba9abdc15d91d26f1e3
Ciphertext 225: 83807c94e05b67ce76501fbd158128b3adce3450f57db473a91ef0e79fb24ec6d1ae945b05b6699ae2b9368d284f3b1083654a4d47dcbdcd08ad83cd14de891bfa9e219043
Ciphertext 226: 10da6efba2e684f648bc3466251d684d3cdc4d64f3c38b691fd1c6451b7257c2bb3b11adc7d876f442b45637174e847b0a862dbab905da393cde2b2c2c87e7c2cb7eb65290
Ciphertext 227: 7c5249c664e9713b9420d07fa475352d07e6b278e30ff855d1037403808415c3d30f6084638253d1873fbf317da32d3334b0f7d2f3cdb186382789ce4eb5e18c7a4642bc95
Ciphertext 228: bb207a80861a3e9b84cce202c0a973237565a2f309ddddc3d745065b5265957e3e0c837e589be67554cd6187b43b46c70b2e9dfff2299a2c280f37ba4f8a28c586612cf236
Ciphertext 229: e12d511131df403d82fcfb6575915ba13da446b485fc644add53f48d5871698376e28409d843acff0c9c2c4a54fe95f2c645c33d6140e26222a16479531d2f00a3d7975369
Ciphertext 230: 50824a99c3e6a82afd44cd6a21eeeb513913d07a87472afd3a8923db957a6f27d0270652834174ecb01be51dffeec38b0f2919ab88c6957a7549148a4d270887b24d3c7b3f
Ciphertext 231: d74f1923072f5babaf8956ce5e69fa9791816e97c901ff442f4b8826ccdf03613541405faba8f513153959cb80e3c9287c6ce9bbecb4090d17e2220d247e1a78a9b4184f39
Ciphertext 232: c112fd5ae94da7b9ecf559c2cd676e03a8be5fd3c77ea11f3acd5888d80c6c4fe7cfc992d19509da1bf29538ec9a15681fe7d2645a7303b95a6528c4fbccb24dff38f48e22
Ciphertext 233: 5bbe654287afb4eb9e01a7ab1ea5b47c70e0beb61a8ec33bdab3fa6a7fd0572795d063b2890f0408669d4763d7b22c80412aa69b9c0639bd2cf7f1efa54a58b3a11baec49b
Ciphertext 234: 2f713cabb39ad20f9b876aacdfbe80b85663ce81a65238a8d890f39c4bd3027ee100720edec714d40422009d6d6d54ab7861bfc1963c7e24be95b055c3c44e8db9a6b8988b
Ciphertext 235: a647b02e962d79fc989214f8e8c57c7b8ced4caf52db8ff46bd97a4fc915c4db30c2b25ba2dbf83e3653ec6032fd8cdf2acc8fe9aa69e7ca9593db00f9480b932f1b8720d9
Ciphertext 236: 15666eec1c5cd916956085d052ad479304ff4577ad0801e5d97e44d63c191b68ef24585916eb4a2f7d5becaae241bcbb206abd45e6d7c1090497ea81e962e220787e7cebef
Ciphertext 237: 9e2c11e84ea9f84630b9b80eb2902c285c65480b2f30ff6515f9195f22d5a3ab97e114b72f7fa5e8c51edb5d4b65874c67df9f596c3dfd8376c350239c9d15b2e6652fc2c1
Ciphertext 238: ea924f4cf49275cf4f4ed977421fc46c37031e7800b5923b4653a17bc9332089b7a6c87c1ecbd53b85ec524d45d3033a2bebe64a8aa3b57078103c01219d0940d790783f2b
Ciphertext 239: 7ebf57bb1472113343dfef606ac32268c1f00146756ae79ccd44efb83b456853f83ea79e4712f338336b1f6b023a1013e6a413d6f92621105cc6114c583bc63dac9ca4f42c
Ciphertext 240: 95020f2becb2e5c965dd0b9fa25e02008af1ae5f3c5266e2de79d6bbdf15240d29a540d0ff48d09abb075892bffc371c345e9e9da226e95b09f3bea59c260e15c41f007c9b
Ciphertext 241: 41b85a09e76c5e3ddbeb8eba6c2638a39b759458f4f1cca4ba0723531c31f2d5042bd2e046ac73a0f1f501bb06f6a9b2582216b2d7b5f148a4997bd73f681abdfd7f473666
Ciphertext 242: d9d46a2bd58b6badfefd69d17d3b2c41d0b1f1c71452d28a130583575943e5093771ad825d9a7996f4e05614c45eca012578973195a9634b86133175f430ba819a4fa9509c
Ciphertext 243: cee1742195ef3b1857556b115184ba2aa4a2147c66bc1120387198c7f1cda7d8024613309b3bcbf60f7f98453f5b1205fbb3a2f23c0c19e375d62e80e082cfcfb48e6e7bc8
Ciphertext 244: 831aa1c1d648606985ff29dc900ffb6cd1b14a6288122f737a91efde54d5171eb21b38ab17032adf964cd10690bde48f279c03f8a422a19ce194ead89e14eb0c66f95eaeef
Ciphertext 245: e7223e54d7046358cdd5bb2b0265732d83ef47fc7590c08863aff89440dbc09e9756875e0cc2ad297524bec38b50c9aaa9af31855ed2df2a742351e14774f6f05617780791
Ciphertext 246: ba3aa9a175ed9278d0db524821f41df5f861600cf2bb03fb5ccd7555f3410e9e47448250bd64b9954b8d10d0ad7c3b7a4b3f7b42f382f0c1bdee5f4cca51241596420129ab
Ciphertext 247: 52ec33732deafbf45fea9cd05e9b45d7fb8e36f2bdaedda2f4fddc7c961c279c71069e5642f13fa40e7146f4010ec75cb19b53715a583a5ce05ba6cbdae7e8802643c0edcd
Ciphertext 248: 75c955fbf3116e2de76055afddf3bb7e1fb219eb521fe757d249453c04973e7dc51bf657f351bfeda3c7e8ab3c77e0483f959ba2c99d5f1d417f83179d3a90ac447d0cff7f
Ciphertext 249: d893656f7e3ef1b0c02585348f177eb4a77efd47d13a5600e159db4365bf8847ec45704c2d603fd8e4955327f44a058767ead74edd4a2cb09e394d5534f85602ba493f9375
Ciphertext 250: 17aa14c9b9f4dd530b57dc42b22a959f9fb0c2b1726c1c0691b601b152f2f72b29890ea996c5368378afa585e7dc70301f0b0034ecd0eb9f406024abc0c5f7d483a31c5127
Ciphertext 251: 2b0f8989dbdfae051e561f81c909a85366879d29c2baa584bfd8bfcf28e957c5e7784ed2cce69ceafb4cab39b76735b3ba0fe12479887934ac21ff36bc7b3cf15e04e2491f
Ciphertext 252: 84ab7f8328697e0bd0aec924096674f35295dd63463d531905e11fcdf075dd76f443b1f949a09fe5ac53907113d502641329f65983932c5c8d8bdb53ea4ca92686b176551d
Ciphertext 253: 82f21a00a53802899de3ee913582aabcfe280007abd87da682283dc58469f69fd9d02db12fcf31dedb634ed07335ed83212a0173528eb1d5e020c3daf7f00489489d05b77a
Ciphertext 254: 0ee458e8a224d0e33314e8df008a2371925ed27af5943fdae82d7d3fd5646bf25ca1acc8760697d61e469349ed88f7ad3c601a84512406f658502ab609ef4b4dfbbb59ca2d
"""

flag = bytes.fromhex("b4a419e77aea51a6a5a6d10d0fe3d57f763387aaeab02295e463c6210a51c3a2505e58ee9be22cf6bc92c64bc214fa5667feb02e88a3dac1a14b706d979ff84b1e42926867")

In [47]:
chosen_plaintext_keystream = bytes([a^b for a,b in zip(flag,b"CSCG")]).hex()
chosen_plaintext_keystream

'f7f75aa0'

In [48]:
# getting rid of \n
key = re.search(f"{chosen_plaintext_keystream}[^\n]*",re.sub("Ciphertext [0-9]{3}: ","",data),re.IGNORECASE).group(0)
key

'f7f75aa001a905f4fae7e25e50a1a74f1d56e995d58f6ca59406ea487e76b0fd136a3482a8867398c9ffa42eb04b8f25029aef41e6c0bf9ec77b0232f6c08a787f31a2061a'

In [52]:
bytes([a^b for a,b in zip(flag,bytes.fromhex(key))])

b"CSCG{CTR_A3S_Br0ken???N0pe,it's_C4ll3d_number_used_once_f0r_a_r3as0n}"